<a href="https://colab.research.google.com/github/JohnDiognes/analise-perfil-eleitor-vila-velha/blob/main/notebooks/01_extracao_e_limpeza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação e Importação de Bibliotecas

In [1]:
!pip install pandas numpy matplotlib seaborn requests zipfile36 py7zr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
import io
import os
import py7zr

pd.set_option("display.max_columns", None)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 29.9 MB/s eta 0:00:00


Download Automático dos Dados do TSE

In [6]:
import os
import requests
import zipfile
import io
import pandas as pd

# Criar pasta de dados se não existir
os.makedirs("dados", exist_ok=True)

# URL correta para o ZIP do eleitorado 2024 no Portal de Dados Abertos do TSE
url = "https://dadosabertos.tse.jus.br/dataset/eleitorado-2024/resource/e35b6053-1dea-4b9f-a4a8-12ae833bf95c/download/perfil_eleitorado_2024.zip"

print("🔽 Baixando ZIP do perfil do eleitorado 2024...")
response = requests.get(url)
if response.status_code != 200:
    print("❌ Erro no download:", response.status_code)
else:
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall("dados")
    print("✔️ Arquivos extraídos na pasta dados/")

# Listar arquivos extraídos para ver o que veio
print("Arquivos extraídos:")
print(os.listdir("dados"))

# Carregar todos os CSVs extraídos (ou pelo menos os que tiverem ES)
for nome in os.listdir("dados"):
    if nome.lower().endswith(".csv"):
        caminho = os.path.join("dados", nome)
        print("Lendo:", nome)
        df = pd.read_csv(caminho, sep=";", encoding="latin1", dtype=str)
        print("Shape:", df.shape)
        print(df.head())

        # Verificar se há dados do Espírito Santo
        if "SIGLA_UF" in df.columns:
            df_es = df[df["SIGLA_UF"] == "ES"]
            print("Registros ES:", df_es.shape)



🔽 Baixando ZIP do perfil do eleitorado 2024...
✔️ Arquivos extraídos na pasta dados/
Arquivos extraídos:
['perfil_eleitor_secao_2024_ES.csv', 'leiame.pdf']
Lendo: perfil_eleitor_secao_2024_ES.csv
Shape: (1573969, 31)
   DT_GERACAO HH_GERACAO ANO_ELEICAO SG_UF CD_MUNICIPIO      NM_MUNICIPIO  \
0  13/09/2024   14:34:53        2024    ES        57053           VITÓRIA   
1  13/09/2024   14:34:53        2024    ES        56294          COLATINA   
2  13/09/2024   14:34:53        2024    ES        56774      NOVA VENÉCIA   
3  13/09/2024   14:34:53        2024    ES        57916  LARANJA DA TERRA   
4  13/09/2024   14:34:53        2024    ES        56111           ARACRUZ   

  NR_ZONA NR_SECAO NR_LOCAL_VOTACAO                      NM_LOCAL_VOTACAO  \
0       1      402             1805   EEEFM MAJOR ALFREDO PEDRO RABAYOLLI   
1       6      271             1449         EMEF PROF. JOÃO ELIAS PANCOTO   
2      30      284             1430  CMEI DOM BOSCO - LUZIA ALVES DE LIMA   
3       7   

Enviar arquivos do seu computador para o Colab

In [17]:
from google.colab import files
uploaded = files.upload()


Saving BairroXRegiao.csv to BairroXRegiao.csv
Saving Local_votacao_Vila_Velha.csv to Local_votacao_Vila_Velha.csv


Mover os arquivos para a pasta Dados

In [21]:
import shutil

# mover arquivos enviados para a pasta dados/
shutil.move("Local_votacao_Vila_Velha.csv", "dados/Local_votacao_Vila_Velha.csv")
shutil.move("BairroXRegiao.csv", "dados/BairroXRegiao.csv")


'dados/BairroXRegiao.csv'

Verificar pasta dados

In [32]:
import os

os.listdir("dados")


['Local_votacao_Vila_Velha.csv',
 'BairroXRegiao.csv',
 'perfil_eleitor_secao_2024_ES.csv',
 'leiame.pdf']

Carregar o arquivo no DataFrame

In [8]:
import pandas as pd

caminho = "dados/perfil_eleitor_secao_2024_ES.csv"

df_es = pd.read_csv(caminho, sep=";", encoding="latin1", dtype=str)

df_es.head()


,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,CD_RACA_COR,DS_RACA_COR,CD_IDENTIDADE_GENERO,DS_IDENTIDADE_GENERO,CD_QUILOMBOLA,DS_QUILOMBOLA,CD_INTERPRETE_LIBRAS,DS_INTERPRETE_LIBRAS,TP_OBRIGATORIEDADE_VOTO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,13/09/2024,14:34:53,2024,ES,57053,VITÓRIA,1,402,1805,EEEFM MAJOR ALFREDO PEDRO RABAYOLLI,4,FEMININO,1,SOLTEIRO,3539,35 a 39 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
1,13/09/2024,14:34:53,2024,ES,56294,COLATINA,6,271,1449,EMEF PROF. JOÃO ELIAS PANCOTO,4,FEMININO,1,SOLTEIRO,4549,45 a 49 anos,2,LÊ E ESCREVE,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
2,13/09/2024,14:34:53,2024,ES,56774,NOVA VENÉCIA,30,284,1430,CMEI DOM BOSCO - LUZIA ALVES DE LIMA,2,MASCULINO,1,SOLTEIRO,3034,30 a 34 anos,6,ENSINO MÉDIO COMPLETO,1,Branca,1,Cisgênero,2,NÃO,2,NÃO,Obrigatório,1,1,0,0
3,13/09/2024,14:34:53,2024,ES,57916,LARANJA DA TERRA,7,135,1040,EMEIF JOÃO VALIM,4,FEMININO,1,SOLTEIRO,3034,30 a 34 anos,5,ENSINO MÉDIO INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
4,13/09/2024,14:34:53,2024,ES,56111,ARACRUZ,20,14,1040,EEEFM DYLIO PENEDO (JACUPEMBA),2,MASCULINO,3,CASADO,4044,40 a 44 anos,5,ENSINO MÉDIO INCOMPLETO,3,Parda,3,Prefere não informar,2,NÃO,2,NÃO,Obrigatório,1,1,0,0


converter colunas para número

In [14]:
cols_numericas = [
    "QT_ELEITORES_PERFIL",
    "QT_ELEITORES_BIOMETRIA",
    "QT_ELEITORES_DEFICIENCIA",
    "QT_ELEITORES_INC_NM_SOCIAL"
]

for col in cols_numericas:
    df[col] = pd.to_numeric(df[col], errors="coerce")


Filtrar somente municipio de Vila Velha

In [25]:
df_vv = df[df["NM_MUNICIPIO"] == "VILA VELHA"]
df_vv.head()



,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,CD_RACA_COR,DS_RACA_COR,CD_IDENTIDADE_GENERO,DS_IDENTIDADE_GENERO,CD_QUILOMBOLA,DS_QUILOMBOLA,CD_INTERPRETE_LIBRAS,DS_INTERPRETE_LIBRAS,TP_OBRIGATORIEDADE_VOTO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
5,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,266,1430,UMEF GRACIANO NEVES,4,FEMININO,3,CASADO,8084,80 a 84 anos,6,ENSINO MÉDIO COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Facultativo,1,1,0,0
11,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,951,2070,COLÉGIO CEIC,4,FEMININO,3,CASADO,6569,65 a 69 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
13,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,936,2070,COLÉGIO CEIC,4,FEMININO,3,CASADO,7074,70 a 74 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Facultativo,1,1,1,0
17,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,57,155,1201,UMEF JUIZ JAIRO DE MATTOS PEREIRA,4,FEMININO,1,SOLTEIRO,5559,55 a 59 anos,4,ENSINO FUNDAMENTAL COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
29,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,57,48,1341,CEEFMTI PASTOR OLIVEIRA DE ARAUJO- ESCOLA VIVA,2,MASCULINO,3,CASADO,6569,65 a 69 anos,6,ENSINO MÉDIO COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0


Visualiza dados

In [26]:
df_vv.groupby("NM_MUNICIPIO").agg({
    "QT_ELEITORES_PERFIL": "sum",
    "QT_ELEITORES_BIOMETRIA": "sum",
    "QT_ELEITORES_DEFICIENCIA": "sum"
}).reset_index()


,NM_MUNICIPIO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA
0,VILA VELHA,345228,317542,9073


Carregar os Arquivos

In [34]:
df_locais = pd.read_csv("dados/Local_votacao_Vila_Velha.csv", encoding="latin-1")
df_bairros = pd.read_csv("dados/BairroXRegiao.csv", encoding="latin-1")

df_locais.head(), df_bairros.head()

(   num_zon nom_localidade  num_local            nom_local  \
 0       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 1       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 2       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 3       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 4       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 
                des_endereco   num_cep nom_bairro  num_secao  qtd_aptos  
 0  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         49        274  
 1  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         50        276  
 2  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         51        276  
 3  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         52        277  
 4  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         53        276  ,
    num_regiao              nom_bairro
 0           1             Boa Vista I
 1           1            Boa Vista II
 2           1                  Centro
 3           1  Coqueiral